# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp
import shutil

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# 3. Extract Keypoint Values

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [5]:
import pandas as pd
import numpy as np
import os
import glob

file_path = 'label.xlsx'
df = pd.read_excel(file_path)
label = df['Label'].to_numpy()
Viet_l = df['Vietnam'].to_numpy()
metric = df[['Label', 'Vietnam']].to_numpy() 

## Rất lưu ý đừng click nhầm!!!!!!!!!!!!!!!!

In [ ]:
for name in label:
    sample = glob.glob('vid/' + name + '/*')
    for idx in range(len(sample)):
        os.rename(sample[idx], 'vid/' + name + '/' + f'{idx}.mp4')

In [ ]:
# convert Viet to Eng
for convert_name in range(len(label)):
    os.rename('vid/' + name[convert_name], 'vid/' + label[convert_name])

In [ ]:
metric.shape

In [ ]:
for idx, name in enumerate(metric):
    if os.path.exists('vid/' + name[1]):
        shutil.rmtree('vid/' + name[1])
    os.makedirs('vid/' + name[1])
    if os.path.exists('keypoint/' + name[0]):
        shutil.rmtree('keypoint/' + name[0])
    os.makedirs('keypoint/' + name[0])
    

In [ ]:
# Video rất thô, nên phải check data, ta có histogram về số frame mỗi vid
# Thời gian bắt đầu và kết thúc của đoạn video bạn muốn giữ lại (ví dụ: 3 giây đầu)
start_time_seconds = 0
end_time_seconds = 3

# Mở video gốc
total = []
for name in label:
    sample = glob.glob('vid/' + name + '/*')
    for idx in range(len(sample)):
        # in_out_video_path = sample[idx]
        
        cap = cv2.VideoCapture(in_out_video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        # frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        
        # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        # out = cv2.VideoWriter(in_out_video_path, fourcc, frame_rate, (frame_width, frame_height))

        total.append(frame_count)

# Giải phóng các tài nguyên
cap.release()
cv2.destroyAllWindows()

In [ ]:
from collections import Counter
total = Counter(total)

In [ ]:
total

In [ ]:
import matplotlib.pyplot as plt
x = total.keys()
y = total.values()
plt.bar(x,y)

# 5. Collect Keypoint Values for Training and Testing

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [8]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [9]:
label

array(['Cabbage', 'Red_pumpkin', 'Tomato', 'Carrot', 'Pea', 'Peanut',
       'Pineapple', 'Sweet_potato', 'Spinach', 'Garlic', 'Corn',
       'Pumpkin', 'Bouquet', 'Locust', 'Flower', 'Bathtub_of_flower',
       'Bamboo_bush', 'Fruit_tree', 'Eagletree', 'Palm', 'Tree',
       'Banyan_Tree', 'Starfruit_tree', 'Maize', 'Phoenix tree',
       'Citronella', 'Woody_plant', 'Pine_tree', 'Bamboo',
       'Bunch_of_flowers', 'Bunch_of_star_fruits', 'Bunch_of_grapes',
       'Bunches_of_fabric', 'Grass', 'Chrysopogon_aciculatus', 'Turnip',
       'Bean_roots', 'Ginger', 'Sweet_potatoes', 'Potato', 'Turmeric',
       'Lemongrass', 'Kohlrabi_bulbs', 'Garlic', 'Yam', 'Stem', 'Petiole',
       'Luffa_trellis', 'Grape_trellis', 'Seed', 'Peanut', 'Grain',
       'Pepper', 'Chrysanthemum', 'Hibiscus_flower', 'Peach_blossom',
       'Confetti', 'Rose', 'Gladiolus', 'Apricot_flowers', 'Lotus',
       'Garden', 'Leaf', 'Paddy', 'Bamboo_pile'], dtype=object)

In [ ]:
def start_stop_estimate(x, limit):
    cut = (1 - limit/x)*x/2
    start = int(cut)
    stop = int(cut + limit)
    while (stop - start) < limit:
        stop += 1
    return(start, stop)

In [ ]:
# setup so keypoints of frame trong 1 vid
sequence_length = 90

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in label:
        # sample la so vid trong 1 class
        # save_dir cung cap vs sample
        sample = glob.glob('vid/' + action + '/*')
        save_dir = 'keypoint/' + action + '/'
        print(len(sample))
        for idx in range(len(sample)):
            cap = cv2.VideoCapture(sample[idx])
            print(sample[idx])
            os.makedirs(save_dir + f'{idx}/')
            
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            start, stop = start_stop_estimate(frame_count, sequence_length)
            for frame_num in range(frame_count):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                if start <= (frame_num) <= stop: 
                    np.save(save_dir + f'{idx}/{frame_num}.npy', keypoints)

# 6. Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
a = ['Bắp cải', 'Bắp ngô', 'Bí đỏ', 'Bí ngô', 'Bó hoa', 'Bồ kết', 'Bồn hoa', 'Bông hoa']
Viet_l

array(['Bắp cải', 'Bí đỏ', 'Cà chua', 'Cà rốt', 'Đậu cô ve', 'Đậu phộng',
       'Dứa', 'Khoai lang', 'Rau muống', 'Tỏi', 'Bắp ngô', 'Bí ngô',
       'Bó hoa', 'Bồ kết', 'Bông hoa', 'Bồn hoa', 'Bụi tre', 'Cây ăn quả',
       'Cây bàng', 'Cây cọ', 'Cây cối', 'Cây đa', 'Cây khế', 'Cây ngô',
       'Cây phượng', 'Cây sả', 'Cây thân gỗ', 'Cây thông', 'Cây tre',
       'Chùm hoa', 'Chùm khế', 'Chùm nho', 'Chùm vải', 'Cỏ', 'Cỏ may',
       'Củ cải trắng', 'Củ đậu', 'Củ gừng', 'Củ khoai lang',
       'Củ khoai tây', 'Củ nghệ', 'Củ sả', 'Củ su hào', 'Củ tỏi', 'Củ từ',
       'Cuống hoa', 'Cuống lá', 'Giàn mướp', 'Giàn nho', 'Hạt', 'Hạt lạc',
       'Hạt lúa', 'Hạt tiêu', 'Hoa cúc', 'Hoa dâm bụt', 'Hoa đào',
       'Hoa giấy', 'Hoa hồng', 'Hoa lay ơn', 'Hoa mai', 'Hoa sen',
       'Khu vườn', 'Lá cây', 'Lúa', 'Luỹ tre'], dtype=object)

In [12]:
b = []
for _, i in enumerate(a): 
    for num, name in enumerate(metric):
        if name[1] == i:
            b.append(name[0])

In [13]:
print(a)
print(b)

['Bắp cải', 'Bắp ngô', 'Bí đỏ', 'Bí ngô', 'Bó hoa', 'Bồ kết', 'Bồn hoa', 'Bông hoa']
['Cabbage', 'Corn', 'Red_pumpkin', 'Pumpkin', 'Bouquet', 'Locust', 'Bathtub_of_flower', 'Flower']


In [14]:
b = np.array(b)

In [15]:
label_map = {label:num for num, label in enumerate(b)}
label_map

{'Cabbage': 0,
 'Corn': 1,
 'Red_pumpkin': 2,
 'Pumpkin': 3,
 'Bouquet': 4,
 'Locust': 5,
 'Bathtub_of_flower': 6,
 'Flower': 7}

In [16]:
sequences, labels = [], []
for name in b:
    sample = glob.glob('keypoint/' + name + '/*')
    for idx in range(len(sample)):
        window = []
        dir = glob.glob(sample[idx] + '/*')
        print(sample[idx])
        for frame_num in range(len(dir)):
            res = np.load(dir[frame_num])
            window.append(res)
        sequences.append(window)
        labels.append(label_map[name])
    print('OK!')

keypoint/Cabbage\0
keypoint/Cabbage\1
keypoint/Cabbage\10
keypoint/Cabbage\11
keypoint/Cabbage\12
keypoint/Cabbage\13
keypoint/Cabbage\14
keypoint/Cabbage\15
keypoint/Cabbage\16
keypoint/Cabbage\17
keypoint/Cabbage\18
keypoint/Cabbage\19
keypoint/Cabbage\2
keypoint/Cabbage\20
keypoint/Cabbage\21
keypoint/Cabbage\22
keypoint/Cabbage\23
keypoint/Cabbage\24
keypoint/Cabbage\25
keypoint/Cabbage\26
keypoint/Cabbage\27
keypoint/Cabbage\28
keypoint/Cabbage\29
keypoint/Cabbage\3
keypoint/Cabbage\4
keypoint/Cabbage\5
keypoint/Cabbage\6
keypoint/Cabbage\7
keypoint/Cabbage\8
keypoint/Cabbage\9
OK!
OK!
keypoint/Red_pumpkin\0
keypoint/Red_pumpkin\1
keypoint/Red_pumpkin\10
keypoint/Red_pumpkin\11
keypoint/Red_pumpkin\12
keypoint/Red_pumpkin\13
keypoint/Red_pumpkin\14
keypoint/Red_pumpkin\15
keypoint/Red_pumpkin\16
keypoint/Red_pumpkin\17
keypoint/Red_pumpkin\18
keypoint/Red_pumpkin\19
keypoint/Red_pumpkin\2
keypoint/Red_pumpkin\20
keypoint/Red_pumpkin\21
keypoint/Red_pumpkin\22
keypoint/Red_pumpkin\

In [17]:
np.array(sequences).shape

(210, 90, 1662)

In [18]:
np.array(labels).shape

(210,)

In [19]:
X = np.array(sequences)

In [20]:
X.shape

(210, 90, 1662)

In [21]:
y = to_categorical(labels).astype(int)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [24]:
X_train.shape

(207, 90, 1662)

In [25]:
y_train.shape

(207, 8)

In [52]:
X_train[0]

array([[ 0.55687153,  0.20235416, -0.70448464, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.55686158,  0.202383  , -0.69348526, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.55763721,  0.20174101, -0.730084  , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.5565412 ,  0.2191357 , -0.65427494, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.55638254,  0.21913479, -0.65245801, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.55762446,  0.20160265, -0.74079216, ...,  0.        ,
         0.        ,  0.        ]])

# 7. Build and Train LSTM Neural Network

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [27]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [42]:
model = Sequential()
model.add(LSTM(16, return_sequences=True, activation='relu', input_shape=(90,1662)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(b.shape[0], activation='softmax'))

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 90, 16)            107456    
                                                                 
 lstm_13 (LSTM)              (None, 90, 64)            20736     
                                                                 
 lstm_14 (LSTM)              (None, 32)                12416     
                                                                 
 dense_12 (Dense)            (None, 128)               4224      
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 8)                 520       
                                                                 
Total params: 153,608
Trainable params: 153,608
Non-tr

In [45]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
7/7 [==============================] - 4s 82ms/step - loss: 3.4601 - categorical_accuracy: 0.1353
Epoch 2/2000
7/7 [==============================] - 1s 83ms/step - loss: 38.8434 - categorical_accuracy: 0.1691
Epoch 3/2000
7/7 [==============================] - 1s 81ms/step - loss: 85.0523 - categorical_accuracy: 0.1884
Epoch 4/2000
7/7 [==============================] - 1s 85ms/step - loss: 224.5887 - categorical_accuracy: 0.1401
Epoch 5/2000
7/7 [==============================] - 1s 84ms/step - loss: 74.3249 - categorical_accuracy: 0.1256
Epoch 6/2000
7/7 [==============================] - 1s 81ms/step - loss: 60.7377 - categorical_accuracy: 0.1546
Epoch 7/2000
7/7 [==============================] - 1s 70ms/step - loss: 51.6920 - categorical_accuracy: 0.1498
Epoch 8/2000
7/7 [==============================] - 0s 70ms/step - loss: 48.7419 - categorical_accuracy: 0.1401
Epoch 9/2000
7/7 [==============================] - 1s 82ms/step - loss: 37.7748 - categorical_accuracy:

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
label[np.argmax(res[5])]

In [ ]:
label[np.argmax(y_test[5])]

# 9. Save Weights

In [47]:
model.save('action_3.h5')

In [ ]:
del model

In [51]:
model.load_weights('action_2.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [48]:
from scipy import stats

In [49]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        # cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [56]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-90:]
        
        if len(sequence) == 90:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(b[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        # #3. Viz logic
        #     if np.unique(predictions[-10:])[0]==np.argmax(res): 
        #         if res[np.argmax(res)] > threshold: 
                    
        #             if len(sentence) > 0: 
        #                 if b[np.argmax(res)] != sentence[-1]:
        #                     sentence.append(b[np.argmax(res)])
        #             else:
        #                 sentence.append(b[np.argmax(res)])

        #     if len(sentence) > 5: 
        #         sentence = sentence[-5:]

        #     # Viz probabilities
        #     image = prob_viz(res, b, image, colors)
            
        # cv2.putText(image, ' '.join(sentence), (3,30), 
        #                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti